# Tracker Requests Positions from Vehicles

## 0.0 - Imports

In [28]:
import pandas as pd 
from requests import post 
import json, pickle

import sqlite3      # to create database
import sqlite_utils # to insert, update and delete information
from sqlalchemy import create_engine

# pandas to transform data and convert to dataframe
# requests.post to invite data to serverless and the serverless response with object
# json to convert data to json or vice versa

### 0.1 - Helper Functions

In [24]:
# Function to get user data and exported to pickle file.
# I deleted data inputed for security, it was only function to now how you done.
def user_logon():
    
    ''' Store the data to Authentication from integration Tracker
        url_logon: url to Authentication and get token
        url_veiculos: Url to load vehicles informations
        url_posicao: Url to load positions informations
        headers: Type of send headers this case is json
        handshake: Data to authentication from integration
                post (informations that you must send like handshake)
                    username: username account to access grid tracker
                    password: password account to access grid tracker
                    appid: Id delivered from Tracker
                response (informations that you receive after logon)
                    token: the alfanumeric code that it's must be send post
                    expiration: There aren't expiration '''
    
    url_logon = 'http://'
    url_veiculos = 'http://'
    url_posicao = 'http://'
    headers = {"content-type":"application/json"}
    handshake = {'username':'', 'password':'', 'appid':'', 'token':'', 'expriration':''}
    
    return url_logon, url_veiculos, url_posicao, headers, handshake

def create_table(cursor):
    
    ''' Create table with SQLAlchemy from DataFrame object'''
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS veiculos (
    id INTEGER, dataCadastrado INTEGER, fuso INTEGER,
    deletado INTEGER, status TEXT, vin TEXT, 
    anoFabricacao REAL, anoModelo REAL, placa TEXT,
    tipoMonitoramento TEXT, marca TEXT, modelo text,
    cor TEXT, descricao TEXT, frota TEXT, tipo TEXT,
    assistencia INTEGER, usuarioCriacao INTEGER,
    proprietarioId INTEGER, proprietario TEXT)
    ''')

### 0.2 - Requests

Requests API from Tracker, with method Post, see the <a href="">documentation</a>

#### 0.2.1 - User Authentication

In [25]:
# Dumping object with data authentication to pickle format
# with open("user_logon.pkl", "wb") as file:
#    pickle.dump(user_logon(), file)


# Loading data from pickle file content user data to connection authentication
with open("user_logon.pkl", "rb") as f_load:
    url_logon, url_vehicles, url_position, headers, handshake = pickle.load(f_load)

Token

In [ ]:
# send data and return object to store token
# without token can't access the vehicle
response_logon = post(url_logon,
                        data=json.dumps(handshake),
                        headers=headers) # response array object
token = {"token":response_logon.json()['object']
        ['token']} # store token form object

#### 0.2.2 - Vehicles

In [ ]:
# Requests to get vehicle informations
response_vehicles = post(url_vehicles, headers=token)
obj_vehicle = response_vehicles.json() # transform obj array to json

In [ ]:
# Save json file in the local folder
# this only for development mode, after validation, it's better maker a 
# function to store object without saving
#with open('collected_2020061810.json', 'w') as json_file:
#    json.dump(obj_veiculo, json_file)

# Transforming Json to DataFrame
obj_vehicle_df = pd.DataFrame(obj_vehicle).copy()

In [ ]:
# Drop List to delete columns that it's not necessary for while
drop_list = """
    codigorf          
    odometroGps        
    dataAquisicao 
    distanciaKmFrete
    kmManual
    horimetroManual
    horimetroAtual
    kmAtual
    statusVenda
    dataAtivado
    dataCancelado
    finalizado
    renavam
    dataInstalacao
    motoristas
    dispositivos
    grupos
    """.split()
df_vehicles = obj_vehicle_df.drop(drop_list, axis=1).copy()

#### 0.2.3 - Positions

In [ ]:
# Requests to get vehicle positions
response_position = post(url_position, headers=token)
obj_position = response_position.json()

In [ ]:
# Transforming Json to DataFrame
df_positions = pd.DataFrame(obj_position).copy()

<h3>Resumo em Português</h3>
<p>realizado o request com método post para a api da tracker no post contém as informações de envio; o tipo de dados "json" os dados do usuário(user, senha, id) e retornado um objecto com as informações e enviadas e mais um objeto com as informações do token, sem o token não é possível fazer a requisição, com isso na requisição para veiculos é enviado o token para que deem acesso aos dados, então retornado um objeto array com os dados dos veiculos, porém mais da metade dos campos(colunas) estão vazias(nulas), com isso uma das informações mais importantes que é o posicionamento do veiculo estão nulas, invalidando o projeto, tem outra url de para requisição que é para pegar especificadamente o posicionamento, porém, retornado vazio também, será comunicado ao responsável técnico da tracker para correção ou permissão da mesma.(20/06/2020)</p>

## 1.0 - Data Description

In [ ]:
# DataFrame from vehicles 
df_vehicles.head()         # see the columns and structure of dataframe

In [ ]:
# Dataframe from positions
df_positions.head()        # see the columns and structure of dataframe

In [ ]:
# MERGE will be a good soluction here? try

df_merge = pd.merge([df_vehicles, df_positions]) # see the index and columns before the execute this

In [ ]:
# after seeing each dataframe, thinking how can you insert data at the database

### 1.1 - Rename Columns

### 1.2 - Data Dimention

### 1.3 - Check NaN and Fillout

In [ ]:
df_merge.isna().sum()

## 2.0 - Data Filtering

In [ ]:
# What columns it's really necessary to store on database.

# List of columns that will be note necessary.
drop_list_merge = [
    
]

df_selected = df_merge.drop(drop_list_merge, axis=1).copy()

## 3.0 - Insert Database

### 3.1 - Connection DataBase

In [ ]:
# Create connection to create_database and table
conn = sqlite3.connect("tracker.db")
cursor = conn.cursor()

# Create engine "connection" from API sqlalchemy to insertion data
engine = create_engine('sqlite:////home/python/PycharmProjects/Projects/tracker/tracker.db', echo=False)

### 3.2 - Create a DataBase

In [ ]:
# Create Table
create_table(cursor)

### 3.3 - DataBase Insertion

In [ ]:
# Run the insertion if is not exists table, else, show the error, remember, 
# the sequence of columns must to be equal by table, else, the information 
# will be change and replaced the wrong place.
df_selected.to_sql('veiculos', con=engine, if_exists='append', index=False)

### 3.2 - DataBase Select Query

In [ ]:
select_cur = pd.DataFrame(cursor.execute(
            'select * from veiculos').fetchall(), columns=obj_df.columns)

## 4.0 - API Request

In [29]:
# API to request data from users